# Finite Difference Heat Equation

The heat equation is a parabolic partial differential equation that describes the distribution of heat (or variation in temperature) in a given region over time.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import *
import pylab as pl
from IPython import display
# import scipy as sc # needed for Alternative way of CN meth
# import scipy.sparse as sparse
# import scipy.sparse.linalg


### Heat system

$\begin{cases} \frac{du}{dt} = k\frac{d^{2}u}{dx^{2}}+f(x,t) \\ u(0, x) = g(x) \\ u(t, 0) = u_0(t) \\ u(t, 1) = u_1(t) \end{cases}$

where $x\in (0, 1), t \in [0, T]$

$u(t, x)-?$

#### Initial parameters:

In [2]:
N, M, T = 100, 100, 5
h = 1/(N-1)
tau = T/(M-1)
x = np.linspace(0, 1, N)
t = np.linspace(0, T, M)
g = 5*np.sin(np.pi*x)
k = 1 # thermal conductivity
u0 = np.zeros_like(t)
u1 = 0.8*t
xv, tv = np.meshgrid(x, t, sparse=True)
f = 2*xv - tv


### Thomas algorithm
Through the solving we will obtain tiagonal matrix which could be easily determined by using Thomas algorithm
refer to http://en.wikipedia.org/wiki/Tridiagonal_matrix_algorithm

so

In [3]:
def TDMAsolver(a, b, c, d):
    nf = len(d) # number of equations
    ac, bc, cc, dc = map(np.array, (a, b, c, d)) # copy arrays
    for it in range(1, nf):
        mc = ac[it-1]/bc[it-1]
        bc[it] = bc[it] - mc*cc[it-1] 
        dc[it] = dc[it] - mc*dc[it-1]
    xc = bc
    xc[-1] = dc[-1]/bc[-1]

    for il in range(nf-2, -1, -1):
        xc[il] = (dc[il]-cc[il]*xc[il+1])/bc[il]

    return xc

### Implicit scheme for heat equation

$\frac{u_i^{j+1}-u_i^j}{\tau}=k\frac{u_{i+1}^{j+1}-2u_i^{j+1}+u_{i-1}^{j+1}}{h^2}+f(t_j, x_i)$

to сanonical form 

$A_iu_{i-1}-B_iu_i+C_iu_{i+1}=D_i$

Source: http://www.polybook.ru/comma/6.4.pdf

In [4]:
def impl_scheme(h, tau, g, k, u0, u1, f):
    M = u0.size
    N = g.size
    u = np.empty((M, N))
    u[0, :] = g
    u[:, 0] = u0
    u[:, N - 1] = u1
    
    for j in range(1, M):
        a = np.ones(N - 2) * k/h**2
        a[0] = 0
        b = np.ones(N - 2) * -(1/tau + 2*k/h**2)
        c = np.ones(N - 2) * k/h**2
        c[-1] = 0
        
        d = -u[j - 1, 1:-1]/tau - f[j, 1:-1]
        d[0] = d[0] - k * u0[j]/h**2
        d[-1] = d[-1] - k * u1[j]/h**2
        
        u[j, 1:-1] = TDMAsolver(a, b, c, d)
        
    return u

### Crank–Nicolson method

$\frac{u_i^{j+1}-u_i^j}{\tau}=\frac{k}{2h^2}((u_{i+1}^{j+1}-2u_i^{j+1}+u_{i-1}^{j+1})+(u_{i+1}^{j}-2u_i^{j}+u_{i-1}^{j})) +f(t_j, x_i)$

which is the same to

$A_iu_{i-1}+B_iu_i+C_iu_{i+1}=D_i$,

where $A_i = C_i = -\frac{k}{2h^2}$, $B_i = \frac{1}{\tau}+\frac{k}{h^2}$, $D_i = -A_iu_{i-1}^j+(\frac{1}{h}+A_i+C_i)u_i^j-C_iu_{i+1}^j +f(t_j, x_i)$

After that determine triagonal matrix by using TDMAsolver

Sorce: http://web.cecs.pdx.edu/~gerry/class/ME448/notes/pdf/CN_slides.pdf

#### Alternative way

According to http://jkwiens.com/2010/01/02/finite-difference-heat-equation-using-numpy/ the other way to determine $u_{i+1}$ is

$(I - \frac{\tau}{2}A)\overline{U_{i+1}} = (I + \frac{\tau}{2}A)\overline{U_i}$,

where $A$ is multiplication of $\frac{k}{\tau^2}$ and $(1,-2,1)$-triagonal matrix, $I$ - indentity one


In [5]:
# but I will try
def crank_nicolson_meth(h, tau, g, k, u0, u1, f):
    M = u0.size
    N = g.size
    u = np.empty((M, N))
    u[0, :] = g
    u[:, 0] = u0
    u[:, N - 1] = u1

    ''' Alternative way
    # Second-Derivative Matrix
    data = np.ones((3, N))
    data[1] = -2 * data[1]
    diags = [-1, 0, 1]
    D2 = k * sparse.spdiags(data, diags, N, N) / (h**2)

    # Identity Matrix
    I = sparse.identity(N)

    for j in range(1, M):
        # Solving the System
        # (I - tau/2 * D2) * u_new = (I + tau/2 * D2) * u_old
        A = (I - tau/2 * D2)
        b = (I + tau/2 * D2) * u[j-1]
        u[j] = sparse.linalg.spsolve(A,  b)
    '''

    for j in range(1, M):
        a = np.ones(N - 2) * -k/(2*h**2)
        a[0] = 0
        b = np.ones(N - 2) * (1/tau + k/h**2)
        c = np.ones(N - 2) * -k/(2*h**2)
        c[-1] = 0
        
        d = -a * u[j-1, 1:-1] + (1/tau + a + c) * u[j-1, 1:-1] - c * u[j-1, 1:-1] + f[j, 1:-1]
        d[0] = d[0] + k * u0[j]/h**2
        d[-1] = d[-1] + k * u1[j]/h**2
        
        u[j, 1:-1] = TDMAsolver(a, b, c, d)
    
    return u

In [6]:
def take_step(iteration):
    pl.rcParams['figure.figsize'] = 10, 7
    pl.xlabel('x', fontsize=24, color='blue')
    pl.ylabel('t°', fontsize=24, color='red')
    pl.xlim([0,1])
    pl.ylim([0,12])
    pl.plot(x, u[iteration])
    display.clear_output(wait=False)
    display.display(pl.gcf())
    plt.clf()


u = impl_scheme(h, tau, g, k, u0, u1, f)
# u = crank_nicolson_meth(h, tau, g, k, u0, u1, f)

interact(take_step, iteration=widgets.IntSlider(min=0,max=99,step=1,value=0)) # won't showing properly on GitHub

<function __main__.take_step>